In [3]:
import numpy as np
import pandas as pd
import math

In [4]:
df = pd.read_csv("streams.csv")
df

,Stream Name,Supply,Target,Cp,U
0,Heated Methanol,43.07,250.0,24.807,1
1,Cooled Product,363.40,89.0,14.101,2
2,Feed To Column-2,156.80,139.0,10.326,1
3,Cooled Water Stream,165.30,30.0,3.420,1
4,DME Final Prouct,45.90,30.0,4.525,2
5,T-100 Condenser,45.90,45.8,20008.333,1
6,T-100 Reboiler,165.70,165.8,15538.889,1
7,T-101 Reboiler,165.30,165.4,23491.667,1
8,T-101 Condenser,126.00,125.9,22375.000,1
9,HU,184.00,183.9,35612.900,1


In [5]:
def is_hot_cold(row):
    if row["Supply"] > row["Target"]:
        return "Hot"
    else:
        return "Cold"
def get_enthalpy(row):
    return (row["Target"] - row["Supply"]) *row["Cp"]

In [6]:
df["Stream Type"] = df.apply(is_hot_cold,axis=1)
df["Delta H"] = df.apply(get_enthalpy,axis=1)
df["Abs Delta H"] = df["Delta H"].apply(abs)
assert abs(df["Delta H"].sum()) < 1, "Process is unbalanced, check that net enthalpy is close to 0"

In [7]:
df

,Stream Name,Supply,Target,Cp,U,Stream Type,Delta H,Abs Delta H
0,Heated Methanol,43.07,250.0,24.807,1,Cold,5133.31251,5133.31251
1,Cooled Product,363.40,89.0,14.101,2,Hot,-3869.31440,3869.31440
2,Feed To Column-2,156.80,139.0,10.326,1,Hot,-183.80280,183.80280
3,Cooled Water Stream,165.30,30.0,3.420,1,Hot,-462.72600,462.72600
4,DME Final Prouct,45.90,30.0,4.525,2,Hot,-71.94750,71.94750
5,T-100 Condenser,45.90,45.8,20008.333,1,Hot,-2000.83330,2000.83330
6,T-100 Reboiler,165.70,165.8,15538.889,1,Cold,1553.88890,1553.88890
7,T-101 Reboiler,165.30,165.4,23491.667,1,Cold,2349.16670,2349.16670
8,T-101 Condenser,126.00,125.9,22375.000,1,Hot,-2237.50000,2237.50000
9,HU,184.00,183.9,35612.900,1,Hot,-3561.29000,3561.29000


In [8]:
cold_df = df[df["Stream Type"] == "Cold"].copy()
hot_df = df[df["Stream Type"] == "Hot"].copy()


cold_df["Low"],cold_df["High"] = cold_df["Supply"],cold_df["Target"]
hot_df["Low"],hot_df["High"] = hot_df["Target"],hot_df["Supply"]

cold_df = cold_df.sort_values(by=["Low","High"])
hot_df = hot_df.sort_values(by=["Low","High"])



In [9]:
cold_df

,Stream Name,Supply,Target,Cp,U,Stream Type,Delta H,Abs Delta H,Low,High
10,CU,5.00,15.0,335.121,1,Cold,3351.21000,3351.21000,5.00,15.0
0,Heated Methanol,43.07,250.0,24.807,1,Cold,5133.31251,5133.31251,43.07,250.0
7,T-101 Reboiler,165.30,165.4,23491.667,1,Cold,2349.16670,2349.16670,165.30,165.4
6,T-100 Reboiler,165.70,165.8,15538.889,1,Cold,1553.88890,1553.88890,165.70,165.8


In [39]:
def inbetween(row,value):
    low = row["Low"]
    high = row["High"]
    c_p = row["Cp"]
    if value >= low and value < high:
        return c_p
    else:
        return 0
def get_regions(df):
    streams = df[["Low","High","Cp"]].copy()
    partitions = df["Low"].tolist()
    partitions.extend(df["High"].tolist())

    partitions = sorted(list(set(partitions)))
    regions = []
    for i,pt in enumerate(partitions[:-1]):
        tot_cp = streams.apply(inbetween, value=pt,axis=1).sum()
        regions.append([pt,partitions[i+1],tot_cp])
    return regions 

In [41]:
get_regions(hot_df)

[[30.0, 45.8, 7.945],
 [45.8, 45.9, 20016.278],
 [45.9, 89.0, 3.42],
 [89.0, 125.9, 17.521],
 [125.9, 126.0, 22392.521],
 [126.0, 139.0, 17.521],
 [139.0, 156.8, 27.847],
 [156.8, 165.3, 17.521],
 [165.3, 183.9, 14.101],
 [183.9, 184.0, 35627.001000000004],
 [184.0, 363.4, 14.101]]

In [36]:
cold_regions = []
cold_df = cold_df.sort_values(by=["Low","High"])



cold_streams = cold_df[["Low","High","Cp"]].copy()
partitions = cold_df["Low"].tolist()
partitions.extend(cold_df["High"].tolist())
partitions = sorted(list(set(partitions)))

for i,pt in enumerate(partitions[:-1]):
    tot_cp = cold_streams.apply(inbetween, value=pt,axis=1).sum()
    cold_regions.append([pt,partitions[i+1],tot_cp])


In [37]:
cold_regions

[[5.0, 15.0, 335.121],
 [15.0, 43.07, 0],
 [43.07, 165.3, 24.807],
 [165.3, 165.4, 23516.474000000002],
 [165.4, 165.7, 24.807],
 [165.7, 165.8, 15563.696],
 [165.8, 250.0, 24.807]]

In [18]:
# Converting to Regions of Heat 

regions = []
delta_T = 10

In [42]:
curr_temp

5.0

In [45]:
i_low

3

In [46]:
cold_regions

[[5.0, 15.0, 335.121], [43.07, 165.3, 24.807], [165.3, 165.7, 23491.667]]